In [ ]:
import pandas as pd
import plotly.express as px

# Charger la base santé
df = pd.read_csv("BASE_FINAL.csv")

# Aperçu
df.head()

# Statistiques descriptives globales
df.describe()

# Boxplot équipements santé par densité
fig = px.box(df, x="Densité_INSEE", y="Equip_Santé_par_1000h",
             points="all", title="Équipements santé pour 1000 habitants par densité")
fig.show()

# Histogramme diversité
fig2 = px.histogram(df, x="Diversité_Equipements", nbins=10,
                    title="Distribution de la diversité des équipements de santé")
fig2.show()

# Communes sous-dotées (top 10)
df.sort_values(by="Equip_Santé_par_1000h").head(10)
